In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import maad
from maad import sound, features, rois
from maad.util import (
    date_parser, plot_correlation_map,
    plot_features_map, plot_features, false_Color_Spectro
    )

In [7]:
SPECTRAL_FEATURES=['MEANf','VARf','SKEWf','KURTf','NBPEAKS','LEQf',
    'ENRf','BGNf','SNRf','Hf', 'EAS','ECU','ECV','EPS','EPS_KURT','EPS_SKEW','ACI',
    'NDSI','rBA','AnthroEnergy','BioEnergy','BI','ROU','ADI','AEI','LFC','MFC','HFC',
    'ACTspFract','ACTspCount','ACTspMean', 'EVNspFract','EVNspMean','EVNspCount',
    'TFSD','H_Havrda','H_Renyi','H_pairedShannon', 'H_gamma', 'H_GiniSimpson','RAOQ',
    'AGI','ROItotal','ROIcover'
    ]

TEMPORAL_FEATURES=['ZCR','MEANt', 'VARt', 'SKEWt', 'KURTt',
    'LEQt','BGNt', 'SNRt','MED', 'Ht','ACTtFraction', 'ACTtCount',
    'ACTtMean','EVNtFraction', 'EVNtMean', 'EVNtCount'
    ]

In [9]:
df = date_parser("Basamba/", dateformat='%Y%m%d_%H%M%S', verbose=True)

# remove index => Date becomes a column instead of an index. This is
# required as df_audio_ind, df_spec_ind and df_spec_ind_per_bin do not have
# date as index. Then we can concatenate all the dataframe.
#df = df.reset_index()

File: SWIFT 02_20220527_004000.wav
File: SWIFT 02_20220527_024000.wav
File: SWIFT 02_20220527_044000.wav
File: SWIFT 02_20220527_064000.wav
File: SWIFT 02_20220527_084000.wav
File: SWIFT 02_20220527_104000.wav
File: SWIFT 02_20220527_124000.wav
File: SWIFT 02_20220527_144000.wav
File: SWIFT 02_20220527_164000.wav
File: SWIFT 02_20220527_184000.wav
File: SWIFT 02_20220527_204000.wav
File: SWIFT 02_20220527_224000.wav
File: SWIFT 02_20220528_004000.wav
File: SWIFT 02_20220528_024000.wav
File: SWIFT 02_20220528_044000.wav
File: SWIFT 02_20220528_064000.wav
File: SWIFT 02_20220528_084000.wav
File: SWIFT 02_20220528_104000.wav
File: SWIFT 02_20220528_124000.wav
File: SWIFT 02_20220528_144000.wav
File: SWIFT 02_20220528_164000.wav
File: SWIFT 02_20220528_184000.wav
File: SWIFT 02_20220528_204000.wav
File: SWIFT 02_20220528_224000.wav
File: SWIFT 02_20220529_004000.wav
File: SWIFT 02_20220529_024000.wav
File: SWIFT 02_20220529_044000.wav
File: SWIFT 02_20220529_064000.wav
File: SWIFT 02_20220

In [ ]:
df_indices = pd.DataFrame()
df_indices_per_bin = pd.DataFrame()

for index, row in df.iterrows() :

    # get the full filename of the corresponding row
    fullfilename = row['file']
    # Save file basename
    path, filename = os.path.split(fullfilename)
    print ('\n**************************************************************')
    print (filename)

    #### Load the original sound (16bits) and get the sampling frequency fs
    try :
        wave,fs = sound.load(filename=fullfilename, channel='left', detrend=True, verbose=False)

    except:
        # Delete the row if the file does not exist or raise a value error (i.e. no EOF)
        df.drop(index, inplace=True)
        continue

    """ =======================================================================
                    Computation in the time domain
    ========================================================================"""

    # Parameters of the audio recorder. This is not a mandatory but it allows
    # to compute the sound pressure level of the audio file (dB SPL) as a
    # sonometer would do.
    S = -35         # Sensbility microphone-35dBV (SM4) / -18dBV (Audiomoth)
    G = 26+16       # Amplification gain (26dB (SM4 preamplifier))

    # compute all the audio indices and store them into a DataFrame
    # dB_threshold and rejectDuration are used to select audio events.
    df_audio_ind = features.all_temporal_alpha_indices(
        s=wave,
        fs=fs,
        gain=G,
        sensibility=S,
        dB_threshold=3,
        rejectDuration=0.01,
        verbose=False,
        display=False
        )

    """ =======================================================================
                    Computation in the frequency domain
    ========================================================================"""

    # Compute the Power Spectrogram Density (PSD) : Sxx_power
    Sxx_power,tn,fn,ext = sound.spectrogram (
        x=wave,
        fs=fs,
        window='hann',
        nperseg=1024,
        noverlap=1024//2,
        verbose=False,
        display=False,
        savefig=None
        )

    # compute all the spectral indices and store them into a DataFrame
    # flim_low, flim_mid, flim_hi corresponds to the frequency limits in Hz
    # that are required to compute somes indices (i.e. NDSI)
    # if R_compatible is set to 'soundecology', then the output are similar to
    # soundecology R package.
    # mask_param1 and mask_param2 are two parameters to find the regions of
    # interest (ROIs). These parameters need to be adapted to the dataset in
    # order to select ROIs
    df_spec_ind, df_spec_ind_per_bin = features.all_spectral_alpha_indices(
        Sxx_power=Sxx_power,
        tn=tn,
        fn=fn,
        flim_low=[0,1500],
        flim_mid=[1500,8000],
        flim_hi=[8000,20000],
        gain=G,
        sensitivity=S,
        verbose=False,
        R_compatible='soundecology',
        mask_param1=6,
        mask_param2=0.5,
        display=False)

    """ =======================================================================
                    Create a dataframe
    ========================================================================"""
    # First, we create a dataframe from row that contains the date and the
    # full filename. This is done by creating a DataFrame from row (ie. TimeSeries)
    # then transposing the DataFrame.
    df_row = pd.DataFrame(row)
    df_row =df_row.T
    df_row.index.name = 'Date'
    df_row = df_row.reset_index()

    # create a row with the different scalar indices
    row_scalar_indices =pd.concat(
        [df_row, df_audio_ind, df_spec_ind],
        axis=1
        )
    # add the row with scalar indices into the df_indices dataframe
    df_indices = pd.concat([df_indices, row_scalar_indices])

    # create a row with the different vector indices
    row_vector_indices = pd.concat(
        [df_row, df_spec_ind_per_bin],
        axis=1)

    # add vector indices into the df_indices_per_bin dataframe
    df_indices_per_bin = pd.concat([df_indices_per_bin, row_vector_indices])

# # Set back Date as index
df_indices = df_indices.set_index('Date')
df_indices_per_bin = df_indices_per_bin.set_index('Date')


**************************************************************
SWIFT 02_20220527_004000.wav


C:\Users\vainq\anaconda3\Lib\site-packages\maad\features\alpha_indices.py:2118: RuntimeWarning: divide by zero encountered in log10
  if log : Sxx = np.log10(Sxx)
C:\Users\vainq\anaconda3\Lib\site-packages\numpy\lib\function_base.py:1452: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])
C:\Users\vainq\anaconda3\Lib\site-packages\maad\util\miscellaneous.py:880: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[  0   0   4 ... 507 509 509]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.update(pd.DataFrame(bbox,
C:\Users\vainq\anaconda3\Lib\site-packages\maad\util\miscellaneous.py:880: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[    7  2316 38936 ... 64833 67262 72244]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.update


**************************************************************
SWIFT13_20220527_004000.wav


C:\Users\vainq\anaconda3\Lib\site-packages\maad\features\alpha_indices.py:2118: RuntimeWarning: divide by zero encountered in log10
  if log : Sxx = np.log10(Sxx)
C:\Users\vainq\anaconda3\Lib\site-packages\numpy\lib\function_base.py:1452: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])
C:\Users\vainq\anaconda3\Lib\site-packages\maad\util\miscellaneous.py:880: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[  0   0   0 ... 509 509 509]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.update(pd.DataFrame(bbox,
C:\Users\vainq\anaconda3\Lib\site-packages\maad\util\miscellaneous.py:880: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[   13 30339 50732 ... 63493 73096 73143]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.update


**************************************************************
SWIFT14_20220527_004000.wav


...


**************************************************************
SWIFT10_20220530_214000.wav


C:\Users\vainq\anaconda3\Lib\site-packages\maad\features\alpha_indices.py:2118: RuntimeWarning: divide by zero encountered in log10
  if log : Sxx = np.log10(Sxx)
C:\Users\vainq\anaconda3\Lib\site-packages\numpy\lib\function_base.py:1452: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])
C:\Users\vainq\anaconda3\Lib\site-packages\maad\util\miscellaneous.py:880: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[  0   0   0 ... 508 509 510]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.update(pd.DataFrame(bbox,
C:\Users\vainq\anaconda3\Lib\site-packages\maad\util\miscellaneous.py:880: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[   12   490  1212 ... 51860 47045  2211]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.update


**************************************************************
SWIFT4_20220530_224000.wav


C:\Users\vainq\anaconda3\Lib\site-packages\maad\features\alpha_indices.py:2118: RuntimeWarning: divide by zero encountered in log10
  if log : Sxx = np.log10(Sxx)
C:\Users\vainq\anaconda3\Lib\site-packages\numpy\lib\function_base.py:1452: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])
C:\Users\vainq\anaconda3\Lib\site-packages\maad\util\miscellaneous.py:880: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[  0  19  20 ... 506 508 509]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.update(pd.DataFrame(bbox,
C:\Users\vainq\anaconda3\Lib\site-packages\maad\util\miscellaneous.py:880: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[   12 14022  7402 ... 73260 70616  1503]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.update


**************************************************************
SWIFT9_20220530_224000.wav


In [ ]:
fig, ax = plot_correlation_map(df_indices, R_threshold=0)

In [ ]:
plot_features_map(df_indices[SPECTRAL_FEATURES], mode='24h')
plot_features_map(df_indices[TEMPORAL_FEATURES], mode='24h')

# A more classical way to analyse variations of indices consists in plotting
# graphs. We choose to normalize rescale their value between 0 to 1 in order to
# compare their trend during a 24h cycle

fig, ax = plt.subplots(3,2, sharex=True, squeeze=True, figsize=(5,5))

fig, ax[0,0] = plot_features(df_indices[['Hf']],norm=True,mode='24h', ax=ax[0,0])
fig, ax[0,1] = plot_features(df_indices[['AEI']],norm=True,mode='24h', ax=ax[0,1])
fig, ax[1,0] = plot_features(df_indices[['NDSI']],norm=True,mode='24h', ax=ax[1,0])
fig, ax[1,1] = plot_features(df_indices[['ACI']],norm=True,mode='24h', ax=ax[1,1])
fig, ax[2,0] = plot_features(df_indices[['TFSD']],norm=True,mode='24h', ax=ax[2,0])
fig, ax[2,1] = plot_features(df_indices[['ROItotal']],norm=True,mode='24h', ax=ax[2,1])

In [ ]:
fcs, triplet = false_Color_Spectro(
    df=df_indices_per_bin,
    indices = [
        'KURTt_per_bin',
        'EVNspCount_per_bin',
        'MEANt_per_bin'
        ],
    reverseLUT=False,
    unit='hours',
    permut=False,
    display=True,
    figsize=(4,7)
    )